In [1491]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [1513]:
#full_training = pd.read_csv('Datasets/crime_merged1.csv')
#full_training = pd.read_csv('Datasets/test_training_merged.csv')
full_training = pd.read_csv('Datasets/clean_merged4.csv')

In [1514]:
full_training = full_training.iloc[:,1:]
full_training.income.describe()
print(full_training.shape)
full_training.head()

(3191, 49)


,state,id,Date,Fatal,SubjectArmed,SubjectRace,SubjectGender,SubjectAge,NatureOfStop,NumberOfShots,...,mainOfficerGender,log_new_age,log_income,scaled_log_new_age,scaled_log_income,scaled_pop,lat,long,state_fatal_prop,city_fatal_prop
0,newmexico,4966058,NaN,F,N,U,M,U,NaN,1,...,M,0.0,9.904737,-0.939642,-0.77168,-0.662392,106.6474,35.1056,1.238095,1.238095
1,newmexico,3154891,2011-10-05,F,Y,L,M,U,NaN,1,...,M,0.0,9.904737,-0.939642,-0.77168,-0.662392,106.6474,35.1056,1.238095,1.238095
2,newmexico,517461,NaN,F,Y,W,F,U,NaN,1,...,M,0.0,9.904737,-0.939642,-0.77168,-0.662392,106.6474,35.1056,1.238095,1.238095
3,newmexico,840208,2010-04-03,N,N,L,U,U,NaN,1,...,M,0.0,9.904737,-0.939642,-0.77168,-0.662392,106.6474,35.1056,1.238095,1.238095
4,newmexico,639066,NaN,F,N,L,M,U,NaN,1,...,M,0.0,9.904737,-0.939642,-0.77168,-0.662392,106.6474,35.1056,1.238095,1.238095


In [1515]:
from sklearn.preprocessing import scale
full_training['scaled_shots'] = scale(full_training.ShotsClean)
full_training['scaled_pop'] = scale(full_training.estimate2016)
full_training['scaled_income'] = scale(full_training.income)
full_training['scaled_age'] = scale(full_training.NewAge)
#full_training['imputed_age'] = full_training.NewAge.replace(0, np.mean(full_training.loc[full_training.NewAge!=0,'NewAge']))
#full_training['imputed_age'] = full_training.NewAge.replace(0,15)

In [1534]:
training_noU = full_training.loc[full_training.Fatal != 'U',:]
# features_list = ['Fatal','SubjectRace', 'SubjectGender', 'SubjectArmed',
#                                'scaled_shots', 'AgeGroup', 'NumberOfOfficers',
#                                    'Month', 'Day', 'scaled_pop', 'mainOfficerRace','income']



features_list2 = ['Fatal','SubjectRace', 'SubjectGender', 'SubjectArmed',
                'NewAge', 'NumberOfOfficers', 'ShotsClean', 'mainOfficerRace',
                  'Year','Month','Day','estimate2016', 'income',
                 'city_fatal_prop']

training_noU['log_age'] = np.log(training_noU.NewAge)
training_noU.loc[training_noU.log_age == -np.inf, 'log_age'] = 0
training_noU['log_pop'] = np.log(training_noU.estimate2016)
training_noU['log_income'] = np.log(training_noU.income)
training_noU['log_shots'] = np.log(np.sqrt(training_noU.ShotsClean))


X = training_noU.loc[:,features_list2].drop('Fatal', axis=1)
X_dummy = pd.get_dummies(X)
y = training_noU.Fatal
print(X_dummy.shape)
print(y.shape)

X_dummy.dropna(axis=0, inplace=True)
X_dummy.shape

(3191, 27)
(3191,)


(3122, 27)

## Splitting into testing and training

In [1517]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X_dummy, y, test_size=0, random_state=12345)

print('train_features dimensions:', train_X.shape)
print('test_features dimensions:', test_X.shape)
print('train_labels dimensions:', train_y.shape)
print('test_labels dimensions:', test_y.shape)
print('The goal is to beat the classification rate of:', y.value_counts(normalize=True)[0])
print(train_y.value_counts(normalize=True))
print(test_y.value_counts(normalize=True))


train_features dimensions: (3191, 27)
test_features dimensions: (0, 27)
train_labels dimensions: (3191,)
test_labels dimensions: (0,)
The goal is to beat the classification rate of: 0.6399743754003844
N    0.639974
F    0.360026
Name: Fatal, dtype: float64
Series([], Name: Fatal, dtype: float64)


In [1518]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier


from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# xgboost causing warnings
import warnings
warnings.filterwarnings('ignore')

In [1519]:
models = []
#models.append(('LogisticRegression', LogisticRegression()))
#models.append(("SVC", SVC(C=100, gamma=0.01)))
#models.append(("KNeighborsClassifier", KNeighborsClassifier(n_neighbors=5)))
models.append(("RandomForestClassifier_gini", RandomForestClassifier(n_estimators=500,
                                                                criterion='gini',
                                                                max_depth=8,
                                                                    max_features=5)))
models.append(("RandomForestClassifier_entropy", RandomForestClassifier(n_estimators=500,
                                                                criterion = 'entropy',
                                                                max_depth=8,
                                                                       max_features=5)))
models.append(('XGBoost', XGBClassifier(learning_rate=0.1, n_estimators=100,
                                       max_depth= 3)))
#models.append(('Adaboost', AdaBoostClassifier(learning_rate=0.1, n_estimators=100)))

results = []
names = []
for name,model in models:
    result = cross_val_score(model, train_X, train_y, cv=5)
    names.append(name)
    results.append(result)

mean_results = []
sd_results = []
for model in results:
    mean_results.append(model.mean())
    sd_results.append(model.std())

results_df = pd.DataFrame({'Model': names,
              'Mean Classification Rate': mean_results,
              'Standard Deviation': sd_results
             })
results_df = results_df.iloc[:,[1,0,2]]

TypeError: '<' not supported between instances of 'float' and 'str'

In [1512]:
results_df.sort_values(by = 'Mean Classification Rate', ascending=False)

,Model,Mean Classification Rate,Standard Deviation
2,XGBoost,0.702207,0.016297
0,RandomForestClassifier_gini,0.701540,0.023820
1,RandomForestClassifier_entropy,0.699877,0.018976


## XGBoost

In [1521]:
from sklearn.metrics import accuracy_score

xgb = XGBClassifier(learning_rate=0.01, n_estimators=500, max_depth=3, subsample = 0.9)
xgb.fit(train_X, train_y)
xgb_pred_class = xgb.predict(test_X)

print('Classification Rate:', accuracy_score(test_y, xgb_pred_class))
pd.crosstab(xgb_pred_class, test_y, margins=False)

TypeError: '<' not supported between instances of 'float' and 'str'

## Random Forest

In [1522]:
rF = RandomForestClassifier(n_estimators=1000, criterion = 'gini', max_depth=9, max_features = 4, oob_score=True)
rF.fit(train_X, train_y)
print('OOB score:', rF.oob_score_)
rF_pred_class = rF.predict(test_X)

print('Classification Rate:', accuracy_score(rF_pred_class, test_y))
print(pd.crosstab(rF_pred_class,test_y))
pd.DataFrame({'Feature': train_X.columns, 'Importances':rF.feature_importances_}).sort_values(by='Importances', ascending=False)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').